In [1]:
import tkinter as tk
from tkinter import messagebox
import mysql.connector
from decimal import Decimal

In [3]:

class ATMInterface:
    def __init__(self, root):
        self.root = root
        self.root.title("ATM Interface")
        self.root.geometry("300x200")
        
        self.initialize_database_connection()
        
        self.account_label = tk.Label(root, text="Account Number:")
        self.account_label.pack()
        self.account_entry = tk.Entry(root)
        self.account_entry.pack()
        
        self.pin_label = tk.Label(root, text="PIN:")
        self.pin_label.pack()
        self.pin_entry = tk.Entry(root, show="*")
        self.pin_entry.pack()
        
        self.login_button = tk.Button(root, text="Login", command=self.login)
        self.login_button.pack()

    def initialize_database_connection(self):
        self.connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password="abhi",
            database="atmdb"
        )

    def login(self):
        account_number = int(self.account_entry.get())
        pin = int(self.pin_entry.get())
        if self.validate_login(account_number, pin):
            self.show_main_menu(account_number)
        else:
            messagebox.showerror("Error", "Invalid login credentials")

    def validate_login(self, account_number, pin):
        try:
            cursor = self.connection.cursor()
            query = "SELECT * FROM accounts WHERE account_number = %s AND pin = %s"
            cursor.execute(query, (account_number, pin))
            result = cursor.fetchone()
            cursor.close()
            return result is not None
        except mysql.connector.Error as e:
            print("Error:", e)
            return False

    def show_main_menu(self, account_number):
        try:
            print("Account number:", account_number)  # Debug statement

            cursor = self.connection.cursor()
            query = "SELECT F_Name, balance FROM accounts WHERE account_number = %s"
            cursor.execute(query, (account_number,))
            result = cursor.fetchone()
            cursor.close()

            print("Result:", result)  # Debug statement

            if result:
                F_Name, balance = result

                self.root.destroy()
                main_menu_root = tk.Tk()
                main_menu_root.title("Main Menu")
                main_menu_root.geometry("300x200")

                welcome_label = tk.Label(main_menu_root, text=f"Welcome {F_Name}")
                welcome_label.pack()

                balance_button = tk.Button(main_menu_root, text="Balance", command=lambda: self.show_balance(account_number))
                balance_button.pack()

                transfer_button = tk.Button(main_menu_root, text="Transfer", command=lambda: self.handle_transfer(account_number))
                transfer_button.pack()

                withdrawal_button = tk.Button(main_menu_root, text="Withdrawal", command=lambda: self.open_withdrawal_window(account_number))
                withdrawal_button.pack()

                last_transaction_button = tk.Button(main_menu_root, text="Last Transaction", command=lambda: self.show_last_transaction(account_number))
                last_transaction_button.pack()

                logout_button = tk.Button(main_menu_root, text="Logout", command=main_menu_root.destroy)
                logout_button.pack()

                main_menu_root.mainloop()
            else:
                messagebox.showerror("Error", "Error retrieving user information.")
        except mysql.connector.Error as e:
            print("MySQL Error:", e)  # Debug statement
            messagebox.showerror("Error", "Error retrieving user information.")

    def open_withdrawal_window(self, account_number):
        self.withdrawal_window = tk.Toplevel()
        self.withdrawal_window.title("Withdrawal")
        self.withdrawal_window.geometry("300x150")

        amount_label = tk.Label(self.withdrawal_window, text="Enter Amount:")
        amount_label.pack()
        self.amount_entry = tk.Entry(self.withdrawal_window)
        self.amount_entry.pack()

        get_amount_button = tk.Button(self.withdrawal_window, text="Get Amount", command=lambda: self.process_withdrawal(account_number))
        get_amount_button.pack()

    def process_withdrawal(self, account_number):
        try:
            amount = Decimal(self.amount_entry.get())  # Convert amount to Decimal type
            cursor = self.connection.cursor()

            # Check if the account has enough balance
            query = "SELECT balance FROM accounts WHERE account_number = %s"
            cursor.execute(query, (account_number,))
            current_balance = cursor.fetchone()[0]
            if current_balance < amount:
                messagebox.showerror("Error", "Insufficient balance for withdrawal")
                return

            # Update balance after withdrawal
            new_balance = current_balance - amount
            update_query = "UPDATE accounts SET balance = %s WHERE account_number = %s"
            cursor.execute(update_query, (new_balance, account_number))

            self.connection.commit()
            messagebox.showinfo("Success", "Withdrawal successful")
            self.withdrawal_window.destroy()  # Close the withdrawal window after successful withdrawal
        except mysql.connector.Error as e:
            print("MySQL Error:", e)
            messagebox.showerror("Error", "Error processing withdrawal")
        except ValueError:
            messagebox.showerror("Error", "Invalid amount")

    def show_balance(self, account_number):
        try:
            cursor = self.connection.cursor()
            query = "SELECT balance FROM accounts WHERE account_number = %s"
            cursor.execute(query, (account_number,))
            balance = cursor.fetchone()[0]
            cursor.close()
            messagebox.showinfo("Balance", f"Your current balance is ${balance}")
        except mysql.connector.Error as e:
            print("Error:", e)
            messagebox.showerror("Error", "Error retrieving balance information.")

    def handle_transfer(self, from_account_number):
        self.transfer_window = tk.Toplevel()
        self.transfer_window.title("Transfer")
        self.transfer_window.geometry("300x150")

        to_account_label = tk.Label(self.transfer_window, text="To Account Number:")
        to_account_label.pack()
        self.to_account_entry = tk.Entry(self.transfer_window)
        self.to_account_entry.pack()

        amount_label = tk.Label(self.transfer_window, text="Amount:")
        amount_label.pack()
        self.amount_entry = tk.Entry(self.transfer_window)
        self.amount_entry.pack()

        transfer_button = tk.Button(self.transfer_window, text="Transfer", command=lambda: self.process_transfer(from_account_number, self.to_account_entry.get(), self.amount_entry.get()))
        transfer_button.pack()

    def process_transfer(self, from_account_number, to_account_number, amount):
        try:
            amount = Decimal(amount)  # Convert amount to Decimal type
            cursor = self.connection.cursor()

            # Check if the sender has enough balance
            query = "SELECT balance FROM accounts WHERE account_number = %s"
            cursor.execute(query, (from_account_number,))
            sender_balance = cursor.fetchone()[0]
            if sender_balance < amount:
                messagebox.showerror("Error", "Insufficient balance for transfer")
                return

            # Update sender's balance
            new_sender_balance = sender_balance - amount
            update_query = "UPDATE accounts SET balance = %s WHERE account_number = %s"
            cursor.execute(update_query, (new_sender_balance, from_account_number))

            # Update receiver's balance
            query = "SELECT balance FROM accounts WHERE account_number = %s"
            cursor.execute(query, (to_account_number,))
            receiver_balance = cursor.fetchone()[0]
            new_receiver_balance = receiver_balance + amount
            update_query = "UPDATE accounts SET balance = %s WHERE account_number = %s"
            cursor.execute(update_query, (new_receiver_balance, to_account_number))

            # Get the next transaction ID
            cursor.execute("SELECT MAX(transaction_id) FROM transactions")
            max_transaction_id = cursor.fetchone()[0]
            if max_transaction_id is None:
                max_transaction_id = 0
            transaction_id = max_transaction_id + 1

            # Insert transaction details into transactions table
            insert_query = "INSERT INTO transactions (transaction_id, From_F_Name, From_account_number, To_F_Name, To_account_number, amount_transferred) VALUES (%s, %s, %s, %s, %s, %s)"
            cursor.execute(insert_query, (transaction_id, self.get_F_Name(from_account_number), from_account_number, self.get_F_Name(to_account_number), to_account_number, amount))

            self.connection.commit()
            messagebox.showinfo("Success", "Transfer successful")
            # Close the transfer window after successful transfer
            self.transfer_window.destroy()
        except mysql.connector.Error as e:
            print("MySQL Error:", e)
            messagebox.showerror("Error", "Error processing transfer")
        except ValueError:
            messagebox.showerror("Error", "Invalid amount")

    def get_F_Name(self, account_number):
        cursor = self.connection.cursor()
        query = "SELECT F_Name FROM accounts WHERE account_number = %s"
        cursor.execute(query, (account_number,))
        result = cursor.fetchone()
        cursor.close()
        return result[0] if result else None

    def show_last_transaction(self, account_number):
        try:
            cursor = self.connection.cursor()
            # Get the last transaction for the logged-in user
            query = "SELECT * FROM transactions WHERE From_account_number = %s OR To_account_number = %s ORDER BY transaction_id DESC LIMIT 1"
            cursor.execute(query, (account_number, account_number))
            result = cursor.fetchone()
            cursor.close()

            if result:
                transaction_id, from_F_Name, from_account_number, to_F_Name, to_account_number, amount_transferred = result
                if from_account_number == account_number:
                    messagebox.showinfo("Last Transaction", f"Amount: {amount_transferred}, Sent to: {to_F_Name} ({to_account_number})")
                else:
                    messagebox.showinfo("Last Transaction", f"Amount: {amount_transferred}, Received from: {from_F_Name} ({from_account_number})")
            else:
                messagebox.showinfo("Last Transaction", "No transactions found")
        except mysql.connector.Error as e:
            print("Error:", e)
            messagebox.showerror("Error", "Error retrieving last transaction information.")

if __name__ == "__main__":
    root = tk.Tk()
    app = ATMInterface(root)
    root.mainloop()


Account number: 123456
Result: ('Abhinav', Decimal('199640.00'))
